## Case 3: 
### Tkintor Demo App For Finding Harmful Ingredients On Cosmetic Products

In [1]:
import pymysql
import pandas as pd
import time
import mysql.connector
import tkinter as tk 
from tkinter import * 
import urllib.request
from PIL import Image, ImageTk
import io
from urllib.request import Request, urlopen

In [ ]:
#Initialize Tkintor 
my_w = tk.Tk()
my_w.geometry("800x400") 

#Connect to GCP project database (Your ip address MUST be registered to access the db and run this application)
my_connect = mysql.connector.connect(
  host="34.133.40.159",
  user="root", 
  passwd="rootroot",
  database="project"
)

###Create connection###
my_conn = my_connect.cursor()

#Assign title to the appp and set the intial text and its position with grid 
my_w.title("Cosmetics App Demo")
lbl = Label(my_w, text="Hello, please type product you are interested in")
lbl.grid(column=1, row=1)

#Create a user input section
txt = Entry(my_w,width=30)
txt.grid(column=2, row=1)

#Function that helps flexbile layout on tkintor app 
def make_label(master, x, y, h, w, *args, **kwargs):
    f = Frame(master, height=h, width=w)
    f.pack_propagate(0) # don't shrink
    f.place(x=x, y=y)
    label = Label(f, *args, **kwargs)
    label.pack(fill=BOTH, expand=1)
    return label

#Function that helps render image on tkintor app 
def ImgFromUrl(url):
    global image
    req = Request(url,headers={'User-Agent': 'XYZ/3.0', "Referer":"https://www.sephora.com"} )
    with urllib.request.urlopen(req, timeout=10) as connection:
        raw_data = connection.read()
    im = Image.open(io.BytesIO(raw_data))
    resize_image = im.resize((100, 100))
    image = ImageTk.PhotoImage(resize_image)
    return image

#Define a function that returns the result of search query upon user input + button click  
def clicked():
    my_conn.execute(f"""SELECT b.brand_name, p.name, h.chemical_name, h.hazard_trait, p.image_url
    FROM products p JOIN product_ingredients i ON p.product_id = i.product_id 
    JOIN harmful_ingredients h ON h.ingredient_id = i.harmful_ingredient_id
    JOIN brands b ON b.brand_index = p.brand
    WHERE p.name LIKE '%{txt.get()}%' AND h.hazard_trait IS NOT NULL LIMIT 1;""")
    result = my_conn.fetchone()
    
    brand = result[0]
    name = result[1]
    chem = result[2]
    harmful = result[3]
    harmfullist = harmful.split(';')
    harmfulingredients = ', '.join(harmfullist)
    url = result[4]
  
    my_w.img = ImgFromUrl(url)
    make_label(my_w, 50, 50, 100, 650, image=my_w.img)
   

    output = f"""You searched for {name} from {brand}. 
    \n It contains {chem} and harmful ingredients such as: 
    \n {harmfulingredients}. \n \n Be cautious before purchasing this product."""
 
    make_label(my_w, 10, 170, 200, 750, text=output, background='light gray')

#Create button and position & connect with function "clicked"
btn = Button(my_w, text="SEARCH", command=clicked)
btn.grid(column=3, row=1)

####### end of connection ####
my_w.mainloop()